In [1]:
import bs4 as bs
from dalab import *
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
def get_links(query):
    query = query.replace(' ', '%20')
    url = 'https://medium.com/search?q=' + query

    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)

    for i in range(0,600):
        driver.execute_script('window.scrollBy(0, 1600)')
        time.sleep(1)
    
    time.sleep(2)

    source = driver.page_source
    soup = bs.BeautifulSoup(source, 'lxml')

    result_list =  soup.find_all('div', {'class': 'js-postListHandle'})[0]
    results = result_list.find_all('div', {'class': 'postArticle-content'})

    links = []
    for result in results:
        links.append(result.a['href'])
        
    driver.close()
    return links

In [ ]:
def get_stories(links):
    stories = []
    for index, link in enumerate(links):
        try:
            soup = get_soup(link)
            title = soup.find_all('h1')[0].text
            body = soup.find_all('div', {'class': 'postArticle-content'})[0]
            story = ''
            for element in body.find_all(recursive=True):
                tag = element.name
                text = element.text
                if tag == 'p':
                    story += ' ' + text
            stories.append(story)
            print(index, 'of', len(links), 'stories')
        except: pass
    return stories

In [ ]:
# queries = ['data science', 'machine learning', 'natural language processing', 'deep learning']

queries = ['artificial intelligence', 'machine learning', 'data science', 'big data', 'neural network', 
           'deep learning', 'natural language processing', 'transfer learning', 'reinforcement learning',
           'nlp', 'recurrent neural', 'convolution neural', 'data mining', 'data extraction', 
           'data engineering', 'web scraping', 'web scrape', 'web crawler', 'web crawling',
           'evolutionary computation', 'genetic algorithm', 'genetic programming', 'object recognition',
           'image understanding', 'speech processing', ' speech recognition', 'machine translation', 
           'time series', 'ai', 'intelligent machine', 'pattern recognition', 'computer vision']

story_dic = dict.fromkeys(queries)

In [ ]:
for query in queries:
    print(' ::', query, ':: ')
    links = get_links(query)
    stories = get_stories(links)
    to_pickle(stories, 'data/medium_stories/' + query.replace(' ', '_'))
    story_dic[query] = stories

df = pd.DataFrame({col: pd.Series(stories) for col, stories in story_dic.items()})
df.columns = [col.replace(' ', '_') for col in df.columns]
df.to_csv('huge_medium.csv')

In [2]:
lst = read_pickle('data/medium_stories/ai.pickle')

In [3]:
lst[0]

' A few months ago, I received a cryptic message from AirBnB that sounded like something straight out that Black Mirror episode with Jon Hamm. At first, I wasn’t concerned. Surely there must be a misunderstanding. After all, I’ve been a loyal AirBnB evangelist from the early days. I referred dozens of friends when it first launched. I even convinced my parents to list their vacation properties. After reaching out to support, I received the following unsettling email. Okay, this is a tough spot. Somehow I’ve violated the terms of service, but they won’t tell me which ones. And I can’t communicate with them anymore. Seems a little harsh. At this point, I was pretty shocked but also very curious about what I could have possibly done. After carefully reading the AirBnB terms of service and reading about the most common ways people get banned from AirBnB, I went through each of my bookings. My first thought was that I must have inadvertently paid one of the hosts in cash because this is the

In [ ]:
# Needs improvement - what tags are relevant to the text body? 
# For now using only text paragraphs (p)
# tag_filter = ['figure', 'img', 'h1', 'h2', 'h3', 'h4', 'h5', 'l1', 'l2', 'l3', 'l4', 'l5' 'strong', 'p', 'figcaption']